<h1> Differential Diagnosis with Mistral 7B RAG vs. BioMistral 7B by ContactDoctor

In [ ]:
!pip install -q streamlit langchain_community chromadb huggingface-hub bitsandbytes pypdf tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.1 MB/s eta 0:00

In [ ]:
import os
os.makedirs('.streamlit', exist_ok=True)
with open('.streamlit/secrets.toml', 'w') as f:
    f.write("""
[huggingface]
token = "secret_token"

[models]
rag = "mistralai/Mistral-7B-Instruct-v0.2"
bio = "BioMistral/BioMistral-7B"
""".lstrip())

In [ ]:
%%writefile app.py

# IMPORT LIBRARY
import streamlit as st
import pandas as pd
import os
import torch

from concurrent.futures import ThreadPoolExecutor, as_completed # FOR PARALLELIZATION

from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain import PromptTemplate
from sentence_transformers import SentenceTransformer

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import threading
import time
from tenacity import retry, stop_after_attempt, wait_fixed
import gc


import logging
from dataclasses import dataclass
from typing import Optional, Dict, Any, List
from contextlib import contextmanager
import psutil
from datetime import datetime

# CONFIGURE LOGGING
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(message)s'
)
logger = logging.getLogger(__name__)



## CODE BLOCK

# GENERAL CONFIGURATION
PROMPT = """Answer the question based only on the following context,:{context}
Question:{question}
What are the top 10 most likely diagnoses? Be precise, listing one diagnosis per line, and try to cover many unique possibilities.
Ensure the order starts with the most likely. The top 10 diagnoses are."""
MAX_INPUT_TOKENS = 2048 # The sequence length limit of BioMistral-7V
MAX_CONTEXT_LENGTH = 4096 # Total context length including prompt
DB_DIR = "./db"

HF_DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PIPELINE_DEVICE = 0 if torch.cuda.is_available() else -1
HF_TOKEN    = st.secrets["huggingface"]["token"]
model_id    = st.secrets["models"]["rag"]
bio_model_id= st.secrets["models"]["bio"]

# using 4bit to save memory for model loading
bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_compute_dtype=torch.bfloat16,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
)


if torch.cuda.is_available():
    import os
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True, max_split_size_mb:512, garbage_collection_threshold:0.8"

# ERROR HANDLING: MESSAGE
class ProcessingError(Exception):
    """Base exception for processing errors"""
    pass

class ModelLoadError(ProcessingError):
    """Error loading or managing models"""
    pass

class TokenLimitError(ProcessingError):
    """Token limit exceeded"""
    pass

class GPUMemoryError(ProcessingError):
    """Memory-related errors"""
    pass

class TimeoutError(ProcessingError):
    """Processing timeout"""
    pass




### HELPERS TO LOAD THE MODEL ###

class ModelManager:
    def __init__(self):
        self.current_model = None
        self.current_model_type = None
        self.gpu_lock = threading.Lock()
        self.rag_components = None
        self.bio_pipeline = None

    def check_gpu_memory(self):
        """Check GPU memory usage"""
        MEMORY_THRESHOLD = 0.9
        if torch.cuda.is_available():
            memory_used = torch.cuda.memory_allocated()
            memory_total = torch.cuda.get_device_properties(0).total_memory
            usage_ratio = memory_used / memory_total
            logger.info(f"GPU memory usage: {usage_ratio:.2%}")

            if usage_ratio > MEMORY_THRESHOLD:
                logger.warning(f"High GPU memory usage: {usage_ratio:.2%}")
                return False
            return True
        return True

    def cleanup_memory(self):
        """Clears CUDA cache and forces garbage collection to save memory"""
        with self.gpu_lock:
          if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
          gc.collect()
          logger.info("Memory cleanup completed")

    def unload_current_model(self):
        """Properly unload current model"""
        with self.gpu_lock:
            if self.current_model:
                try:
                    if hasattr(self.current_model, 'model') and hasattr(self.current_model.model, 'to'):
                        self.current_model.model.to('cpu')
                    del self.current_model
                    self.current_model = None
                    self.current_model_type = None
                    self.cleanup_memory()
                    logger.info("Model unloaded successfully")
                except Exception as e:
                    logger.error(f"Error unloading model: {e}")


    def get_rag_components(self):
        """Get RAG components with proper management"""
        if self.rag_components is None:
            model = None
            pipe = None
            try:
                logger.info("Loading RAG components...")
                embedding = HuggingFaceEmbeddings(
                    model_name="sentence-transformers/all-mpnet-base-v2",
                    model_kwargs={"device": HF_DEVICE}
                )

                vs = Chroma(
                    embedding_function=embedding,
                    persist_directory=DB_DIR,
                )
                retriever = vs.as_retriever()

                model = AutoModelForCausalLM.from_pretrained(
                    model_id,
                    use_auth_token=HF_TOKEN,
                    device_map='auto' if HF_DEVICE == "cuda" else "cpu",
                    torch_dtype=torch.bfloat16 if HF_DEVICE == "cuda" else torch.float32,
                    quantization_config=bnb_config
                )

                pipe = pipeline(
                    "text-generation",
                    model=model,
                    tokenizer=AutoTokenizer.from_pretrained(model_id, use_auth_token=HF_TOKEN),
                    use_fast=True,
                    max_new_tokens=256,
                )

                if HF_DEVICE == "cuda":
                    pipe.model.to("cuda")

                prompt = PromptTemplate(template=PROMPT, input_variables=["context", "question"])
                self.rag_components = (pipe, retriever, prompt)
                logger.info("RAG components loaded successfully")

            except Exception as e:
              if pipe is not None:
                try:
                    if hasattr(pipe, 'model') and HF_DEVICE == "cuda":
                        pipe.model.to("cpu")
                    del pipe
                except:
                    pass
            if model is not None:
                try:
                    if HF_DEVICE == "cuda":
                        model.to("cpu")
                    del model
                except:
                    pass
            self.cleanup_memory()
            logger.error(f"Error loading RAG components: {e}")
            raise ModelLoadError(f"Failed to load RAG components: {e}")

        return self.rag_components

    def get_bio_pipeline(self):
        """Get Bio pipeline with proper management"""
        if self.bio_pipeline is None:
            try:
                logger.info("Loading Bio pipeline...")
                with self.gpu_lock:
                    bio_model = AutoModelForCausalLM.from_pretrained(
                        bio_model_id,
                        use_auth_token=HF_TOKEN,
                        device_map='auto' if HF_DEVICE == "cuda" else "cpu",
                        torch_dtype=torch.bfloat16 if HF_DEVICE == "cuda" else torch.float32,
                        quantization_config=bnb_config
                    )

                    self.bio_pipeline = pipeline(
                        "text-generation",
                        model=bio_model,
                        tokenizer=AutoTokenizer.from_pretrained(bio_model_id, use_auth_token=HF_TOKEN),
                        use_fast=True,
                        max_new_tokens=256,
                    )

                    if HF_DEVICE == "cuda":
                        self.bio_pipeline.model.to("cuda")

                logger.info("Bio pipeline loaded successfully")
            except Exception as e:
                logger.error(f"Error loading Bio pipeline: {e}")
                raise ModelLoadError(f"Failed to load Bio pipeline: {e}")

        return self.bio_pipeline

# Initialize model manager
model_manager = ModelManager()



### CACHING HEAVY RESOURCES ###

@st.cache_data(show_spinner=False)
def build_vectorstore(uploaded_files, DB_DIR ="./db"):
    """Build vector store with improved error handling"""
    try:
        logger.info(f"Building vector store from {len(uploaded_files)} files")
        uploaded_textbook = "./uploaded_textbook"

        # Create directories
        os.makedirs(uploaded_textbook, exist_ok=True)
        os.makedirs(DB_DIR, exist_ok=True)

        # Save uploaded files
        paths = []
        for f in uploaded_files:
            path = os.path.join(uploaded_textbook, f.name)
            with open(path, "wb") as fp:
                fp.write(f.getbuffer())
            paths.append(path)
            logger.info(f"Saved file: {f.name}")

        # Load documents
        docs = []
        for pdf_path in paths:
            try:
                pdf_docs = PyPDFLoader(pdf_path).load()
                docs.extend(pdf_docs)
                logger.info(f"Loaded {len(pdf_docs)} pages from {pdf_path}")
            except Exception as e:
                logger.error(f"Error loading {pdf_path}: {e}")
                st.error(f"Error loading {pdf_path}: {e}")

        if not docs:
            raise ProcessingError("No documents were successfully loaded")

        # Split documents
        splitter = CharacterTextSplitter.from_tiktoken_encoder(
            chunk_size=1000, chunk_overlap=150
        )
        splits = splitter.split_documents(docs)
        logger.info(f"Created {len(splits)} document chunks")

        # Create vector store
        embedding = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-mpnet-base-v2",
            model_kwargs={"device": HF_DEVICE}
        )

        vs = Chroma.from_documents(
            splits,
            embedding,
            persist_directory=DB_DIR,
        )
        vs.persist()
        logger.info("Vector store built and persisted successfully")
        return True

    except Exception as e:
        logger.error(f"Error building vector store: {e}")
        st.error(f"Error building vector store: {e}")
        return False






### HELPERS ###

@contextmanager
def model_context(model_type):
    """Context manager for safe model usage"""
    try:
        if model_type == "rag":
            model = model_manager.get_rag_components()[0]  # Get pipeline
        elif model_type == "bio":
            model = model_manager.get_bio_pipeline()
        else:
            raise ValueError(f"Unknown model type: {model_type}")

        if HF_DEVICE == "cuda":
            with model_manager.gpu_lock:
                model.model.to("cuda")

        yield model

    finally:
        if HF_DEVICE == "cuda":
            with model_manager.gpu_lock:
                if hasattr(model, 'model'):
                    model.model.to("cpu")
                model_manager.cleanup_memory()

def health_check():
    """Perform system health check"""
    checks = {
        'timestamp': datetime.now().isoformat(),
        'gpu_available': torch.cuda.is_available(),
        'gpu_memory_ok': model_manager.check_gpu_memory() if torch.cuda.is_available() else True,
        'index_exists': os.path.exists(DB_DIR) and bool(os.listdir(DB_DIR)) if os.path.exists(DB_DIR) else False,
        'cpu_usage': psutil.cpu_percent(interval=None),
        'memory_usage': psutil.virtual_memory().percent
    }
    logger.info(f"Health check: {checks}")
    return checks

@st.cache_resource(show_spinner=False)
def get_tokenizer():
    """Get tokenizer with caching"""
    try:
        return AutoTokenizer.from_pretrained(model_id, use_auth_token=HF_TOKEN)
    except Exception as e:
        logger.error(f"Error loading tokenizer: {e}")
        raise ModelLoadError(f"Failed to load tokenizer: {e}")

def check_length(text, tokenizer=None):
    """Check if text length is within limits"""
    try:
        if tokenizer is None:
            tokenizer = get_tokenizer()
        token_count = len(tokenizer.encode(text))
        logger.info(f"Input token count: {token_count}")

        if token_count > MAX_INPUT_TOKENS:
            error_msg = f"Input is {token_count} tokens, over the {MAX_INPUT_TOKENS}-token limit"
            logger.warning(error_msg)
            st.warning(error_msg + ". Please shorten it.")
            raise TokenLimitError(error_msg)
        return True
    except Exception as e:
        logger.error(f"Error checking text length: {e}")
        return False

def check_context_length(prompt_text, tokenizer=None):
    """Check if context length is within limits"""
    try:
        if tokenizer is None:
            tokenizer = get_tokenizer()
        token_count = len(tokenizer.encode(prompt_text))
        logger.info(f"Context token count: {token_count}")

        if token_count > MAX_CONTEXT_LENGTH - 256:
            error_msg = f"Total context + prompt is {token_count} tokens, over the {MAX_CONTEXT_LENGTH}-token limit"
            logger.warning(error_msg)
            st.warning(error_msg)
            raise TokenLimitError(error_msg)
        return True
    except Exception as e:
        logger.error(f"Error checking context length: {e}")
        return False

def safe_invoke(model_or_chain, *args, **kwargs):
    """Safely invoke model with error handling"""
    try:
        if hasattr(model_or_chain, "invoke"):
            result = model_or_chain.invoke(*args, **kwargs)
        else:
            result = model_or_chain(*args, **kwargs)
        return result
    except Exception as e:
        logger.error(f"Model invocation error: {e}")
        st.error(f"MODEL ERROR: {e}")
        raise ProcessingError(f"Model invocation failed: {e}")


#error handling for bio and naive model
@retry(stop=stop_after_attempt(2), wait=wait_fixed(2))
def run_rag_pipeline(prompt_text):
    """Run RAG pipeline with improved error handling"""
    try:
        logger.info("Running RAG pipeline")
        with model_context("rag") as pipe:
            raw_output = safe_invoke(pipe, prompt_text, max_new_tokens=256)

            if raw_output and len(raw_output) > 0:
                raw = raw_output[0]["generated_text"]
                output = raw[len(prompt_text):].lstrip() if raw.startswith(prompt_text) else raw
                logger.info("RAG pipeline completed successfully")
                return output
            else:
                raise ProcessingError("No output generated from RAG pipeline")

    except Exception as e:
        logger.error(f"RAG pipeline error: {e}")
        raise ProcessingError(f"RAG pipeline failed: {str(e)}")

@retry(stop=stop_after_attempt(2), wait=wait_fixed(2))
def run_bio_pipeline(prompt_text):
    """Run Bio pipeline with improved error handling"""
    try:
        logger.info("Running Bio pipeline")
        with model_context("bio") as pipe:
            raw_output = safe_invoke(pipe, prompt_text, max_new_tokens=256)

            if raw_output and len(raw_output) > 0:
                raw = raw_output[0]["generated_text"]
                output = raw[len(prompt_text):].lstrip() if raw.startswith(prompt_text) else raw
                logger.info("Bio pipeline completed successfully")
                return output
            else:
                raise ProcessingError("No output generated from Bio pipeline")

    except Exception as e:
        logger.error(f"Bio pipeline error: {e}")
        raise ProcessingError(f"Bio pipeline failed: {str(e)}")


#case processing
def process_case(txt, use_rag=True, use_bio=True):
    """Process a single case with comprehensive error handling"""
    results = {"Case": txt[:100] + "..." if len(txt) > 100 else txt}
    logger.info(f"Processing case of length {len(txt)}")

    try:
        tokenizer = get_tokenizer()

        # Validate input length
        if not check_length(txt, tokenizer):
            error_msg = "Case too long, exceeds token limit"
            results["Mistral7B+RAG"] = error_msg
            results["BioMistral7B"] = error_msg
            return results

        # Process with RAG
        if use_rag:
            try:
                logger.info("Starting RAG processing")
                _, retriever, prompt_template = model_manager.get_rag_components()

                # Retrieve context
                docs = retriever.get_relevant_documents(txt)
                context = "\n\n".join(d.page_content for d in docs)

                # Generate prompt
                prompt_text = prompt_template.format_prompt(context=context, question=txt).to_string()

                # Check context length and truncate if needed
                try:
                    check_context_length(prompt_text, tokenizer)
                except TokenLimitError:
                    logger.warning("Context too long, truncating")
                    context_shortened = context[:len(context)//2] + "..."
                    prompt_text = prompt_template.format_prompt(context=context_shortened, question=txt).to_string()

                # Run RAG pipeline
                rag_out = run_rag_pipeline(prompt_text)
                results["Mistral7B+RAG"] = rag_out
                logger.info("RAG processing completed")

            except Exception as e:
                error_msg = f"RAG ERROR: {str(e)}"
                logger.error(error_msg)
                results["Mistral7B+RAG"] = error_msg
        else:
            results["Mistral7B+RAG"] = "RAG processing skipped"

        # Process with Bio model
        if use_bio:
            try:
                logger.info("Starting Bio processing")
                bio_prompt = PROMPT.format(context="", question=txt)
                bio_out = run_bio_pipeline(bio_prompt)
                results["BioMistral7B"] = bio_out
                logger.info("Bio processing completed")

            except Exception as e:
                error_msg = f"BIO ERROR: {str(e)}"
                logger.error(error_msg)
                results["BioMistral7B"] = error_msg
        else:
            results["BioMistral7B"] = "Bio processing skipped"

    except Exception as e:
        error_msg = f"GENERAL ERROR: {str(e)}"
        logger.error(error_msg)
        results["Mistral7B+RAG"] = error_msg
        results["BioMistral7B"] = error_msg

    return results



### STREAMLIT UI ###

st.title("Differential Diagnosis: Mistral 7B RAG vs. Bio Mistral 7B by BioMistral")
st.caption("Helps the doctor/nurse to develop their differential diagnosis using LLM models")
st.caption("Pretrained Medical Textbook: ....")


# Additional files
with st.sidebar:
    st.header("Upload additional resources for RAG (type:.pdf)")
    UploadedFiles = st.file_uploader("Upload here and click on 'Upload'", type="pdf", accept_multiple_files=True)
    MAX_LINES = 3 # limit maximum document uploaded
    if len(UploadedFiles) > MAX_LINES:
      st.warning(f"Maximum number of files reached. Only the first {MAX_LINES} will be processed.")
      UploadedFiles = UploadedFiles[:MAX_LINES]


    if st.button("Build Index"):
        if not UploadedFiles:
            st.error("Select at least one PDF first.")
        else:
            with st.spinner("Indexing…"):
                build_vectorstore(UploadedFiles, DB_DIR)
            st.success("RAG index is ready!")

    st.markdown("---")
    st.header("System Status")
    if st.button("Check System Health"):
        health = health_check()
        for key, value in health.items():
            if key == 'timestamp':
                st.text(f"Last check: {value}")
            elif isinstance(value, bool):
                st.text(f"{key}: {'✅' if value else '❌'}")
            elif isinstance(value, (int, float)):
                st.text(f"{key}: {value:.1f}%")
            else:
                st.text(f"{key}: {value}")

    if st.button("Clear GPU Memory"):
        model_manager.cleanup_memory()
        st.success("GPU memory cleared!")

    st.markdown("---")
    st.header("Batch processing case upload (type:.xlsx)")
    excel_file = st.file_uploader(
        "Upload .XLSX",
        type="xlsx",
        accept_multiple_files=False)

### SINGLE CASE ###

st.subheader("SINGLE CASE")
question = st.text_area("Case Narrative:",
                        height=180,
                        placeholder="For example: 22-year-old patient with TB was admitted to hospital today. The patient has been to a country outside Sweden. The patient came back to Sweden from the other country. The patient has had a fever for two weeks and is admitted. The doctor has prescribed a medicine. ")
st.write(f"The number of characters are {len(question)} characters.")



if st.button('Start Processing'):
    # Check system health
    health = health_check()
    if not health['index_exists']:
        st.error("Please upload and build your PDF index first!")
        st.stop()

    if not health['gpu_memory_ok']:
        st.warning("High GPU memory usage detected. Processing may be slower.")

    try:
        check_length(question)
        with st.spinner("Processing..."):
            result = process_case(question)

        tabs = st.tabs(["BIOMode", "RAGMode", "System Info"])

        with tabs[0]:
            st.markdown("**BioMistral 7B**")
            st.text(result['BioMistral7B'])

        with tabs[1]:
            st.markdown("**Mistral 7B + RAG**")
            st.text(result['Mistral7B+RAG'])

        with tabs[2]:
            st.markdown("**System Status**")
            st.json(health_check())

    except TokenLimitError as e:
        st.error(f"Input too long: {e}")
    except Exception as e:
        st.error(f"Processing error: {e}")
        logger.error(f"UI processing error: {e}")

### BATCH PROCESSING ###

def process_batch_safely(cases, progress_bar=None):
    """Process batch of cases with proper error handling"""
    BATCH_SIZE = 5
    results = []
    total_cases = len(cases)
    logger.info(f"Starting batch processing of {total_cases} cases")

    # Process in smaller batches to manage memory
    for i in range(0, total_cases, BATCH_SIZE):
        batch = cases[i:i+BATCH_SIZE]
        logger.info(f"Processing batch {i//BATCH_SIZE + 1}")

        for case_idx, case in enumerate(batch):
            try:
                # Add validation
                if not case or not case.strip():
                    logger.warning(f"Skipping empty case {i + case_idx + 1}")
                    error_result = {
                        "Case": "Empty case",
                        "Mistral7B+RAG": "ERROR: Empty case provided",
                        "BioMistral7B": "ERROR: Empty case provided"
                    }
                    results.append(error_result)
                    if progress_bar:
                        progress_bar.progress((i + case_idx + 1) / total_cases)
                    continue

                # Check system health before processing
                health = health_check()
                if not health['gpu_memory_ok']:
                    logger.warning("GPU memory high, forcing cleanup")
                    model_manager.cleanup_memory()

                result = process_case(case)
                results.append(result)

                # Update progress bar if provided
                if progress_bar:
                    progress_bar.progress((i + case_idx + 1) / total_cases)

                logger.info(f"Completed case {i + case_idx + 1}/{total_cases}")

            except Exception as e:
                error_result = {
                    "Case": case[:100] + "..." if len(case) > 100 else case,
                    "Mistral7B+RAG": f"ERROR: {str(e)}",
                    "BioMistral7B": f"ERROR: {str(e)}"
                }
                results.append(error_result)

                if progress_bar:
                    progress_bar.progress((i + case_idx + 1) / total_cases)

                logger.error(f"Error processing case {i + case_idx + 1}: {e}")

        # Cleanup between batches
        model_manager.cleanup_memory()
        logger.info(f"Completed batch {i//BATCH_SIZE + 1}")

    logger.info(f"Batch processing completed. Processed {len(results)} cases")
    return results



st.markdown("---")
st.subheader("BATCH MODE")

if excel_file:
    df = pd.read_excel(excel_file)

    if st.button("Start Batch Processing"):
      health = health_check()
      if not health['index_exists']:
          st.error("Please upload and build your PDF index first!")
          st.stop()

      if not health['gpu_memory_ok']:
          st.warning("High GPU memory usage detected. Batch processing may be slower.")

      try:
          if 'Case' not in df.columns:
            st.error("Excel file must contain a 'Case' column!")
            st.stop()
          cases = df["Case"].tolist()
          prog = st.progress(0)

          # Use the defined batch processing function
          results = process_batch_safely(cases, prog)

      try:
          out_df = pd.DataFrame(results)
          st.download_button(
              "Download Results as CSV",
              data=out_df.to_csv(index=False),
              file_name="ddx_comparison.csv",
              mime="text/csv"
          )
          logger.info("CSV download prepared successfully")
      except Exception as e:
          st.error(f"Error preparing CSV download: {e}")
          logger.error(f"CSV generation error: {e}")

          # Show sample of results
          st.write("Sample of processed results:")
          st.dataframe(out_df.head())

          logger.info("Batch processing completed")

      except pd.errors.EmptyDataError:
          st.error("Excel file is empty!")
      except pd.errors.ParserError:
          st.error("Error reading Excel file. Please check the format.")
      except KeyError as e:
          st.error(f"Missing column in Excel file: {e}")
      except Exception as e:
          st.error(f"Batch processing error: {e}")
          logger.error(f"Batch processing error: {e}")

Writing app.py


<h2>Install local-tunnel </h2>

In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
added 22 packages in 3s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏

<h2> Run Streamlit in background </h2>

In [ ]:
# AND Expose to the port 8501
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.187.241.161
⠙your url is: https://tired-groups-clap.loca.lt
